# Зависимости

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from mycsv import split_file
from dataset import load_dataset
from transform import Converter

# Загрузка модели

In [ ]:
MODEL_PATH = 'fnp_lstm_maxlen200_e25_p701.h5'
model = load_model(MODEL_PATH)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           316000    
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          82432     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 126)               16254     
Total params: 546,270
Trainable params: 546,270
Non-trainable params: 0
_________________________________________________________________


# Загрузка данных для исследования

In [ ]:
split_file('dataset.csv', split_ratio=0.8)

После разделения датасета в дериктории появиться два файла (тренировочный сет и тестовый).

* `maxlen` задает максимальную длину последовательности узлов функции (средняя длина последовательности ~50)

* `verbose` задает многословность функций обработчиков (то столько будет выводится информации во время обратоки); доступные значения от 0 до 3

In [ ]:
train_file_name = 'train_dataset.csv'
test_file_name = 'test_dataset.csv'
maxlen = 200
verbose = 0

Загрузка конвертеров из файлов

In [ ]:
data_converter = Converter.from_file('dictionary_9875.json')
marks_converter = Converter.from_file('dictionary_126.json')

Загрузка данных для исследования

In [ ]:
dataset = load_dataset(train_file_name, 
                       test_file_name, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_train, y_train, x_test, y_test, data_converter, marks_converter = dataset

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Доля верных ответов на тестовых данных составляет:',round(scores[1]* 100, 4),'%')

1034/1034 [==============================] - 123s 119ms/step - loss: 1.3214 - accuracy: 0.7013
Доля верных ответов на тестовых данных составляет: 70.1255 %


# Исследование работы модели на исходном датасете

# Исследование работы модели на реальных примерах